In [1]:
import nltk
from collections import Counter
import glob
import os
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Paths for all the different corpora
folderpaths = ['../AnxietyMeditations/', '../SleepMeditations/',
               '../LearningAndGrowthMeditations/', '../MorningMeditations/']

#array to store the text of each file 
corpus = []

for folder in folderpaths:
    if folder == '../AnxietyMeditations/':
        for doc in glob.glob(os.path.join(folder, '*.txt')):
            with open(doc, 'r') as f:
                text = f.read()
                corpus.append(text)
    elif folder == '../SleepMeditations/':
        for doc in glob.glob(os.path.join(folder, '*.txt')):
            with open(doc, 'r') as f:
                text = f.read()
                corpus.append(text)
    elif folder == '../LearningAndGrowthMeditations/':
        for doc in glob.glob(os.path.join(folder, '*.txt')):
            with open(doc, 'r') as f:
                text = f.read()
                corpus.append(text)
    elif folder == '../MorningMeditations/':
        for doc in glob.glob(os.path.join(folder, '*.txt')):
            with open(doc, 'r') as f:
                text = f.read()
                corpus.append(text)

newStopWords = ['[', ']', '`', '#', '$',
                '(', ')', "Music", ':', "....", "--", ",", "''", '`', '"']
stopwords = set(stopwords.words('english') +
                newStopWords + list(string.punctuation))

# Tokenizing by word and filtering stopwords
all_tokens = [w for w in word_tokenize(text.lower()) if not w in stopwords]



In [2]:
import nltk
from nltk.tokenize import word_tokenize

# Preprocess the text data
tokenized_corpus = []
for text in corpus:
    # Tokenize and lowercase
    tokens = [w for w in word_tokenize(text.lower()) if not w in stopwords]
    tokenized_corpus.append(tokens)


In [9]:
import os
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
# Preprocess the text data
tokenized_corpus = []
for text in corpus:
    tokens = word_tokenize(text.lower())  # Tokenize and lowercase
    tokenized_corpus.append(tokens)

context_windows = []
window_size = 4  # Size of the context window

for sentence in tokenized_corpus:
    sentence_length = len(sentence)
    for i in range(sentence_length):
        # Extract the target word
        target_word = sentence[i]

        # Extract the context words within the window size
        context_words = []
        for j in range(max(0, i - window_size), min(sentence_length, i + window_size + 1)):
            if j != i:
                context_words.append(sentence[j])

        # Append the target word and its context words as a tuple
        # Convert context_words to tuple
        context_windows.append((target_word, tuple(context_words)))

# Train the Word2Vec model
model = Word2Vec(context_windows, window=5,
                 min_count=10, sg=0)  # CBOW approach

# Accessing trained word vectors
word_vector = model.wv['breathe']  # Get the word vector for a specific word
print(word_vector)

[-0.05216778  0.04700494  0.00467626  0.11602146  0.06161752 -0.05302056
  0.09786946  0.06228751 -0.11274248 -0.00464894 -0.00156548 -0.11005984
  0.01198301 -0.03416103 -0.01878685  0.05118901  0.00337715 -0.10038635
 -0.02047516 -0.23186155 -0.02321271  0.10729339  0.11130142 -0.06880401
 -0.01948138  0.08279314 -0.05761955 -0.00093203 -0.13145903  0.00318746
  0.08309339  0.04528908  0.06778198  0.01051673 -0.11951401  0.11049
 -0.00174817  0.00416544 -0.06486303 -0.13929689  0.10906863 -0.01826561
 -0.02472208 -0.07160486  0.08340048 -0.02346409 -0.09259065  0.07561697
  0.11196625  0.03228802 -0.01867998 -0.01492263  0.0291295   0.0139758
 -0.06425641  0.06143889 -0.01537704 -0.10174065 -0.01343953 -0.04914239
  0.03370908  0.05734982  0.09711124  0.0171025  -0.05078335  0.02788507
 -0.12220423  0.019498   -0.11136904 -0.02480489 -0.01029641  0.09406907
  0.00445146 -0.03897272  0.03305973 -0.03689892  0.03455167 -0.10253716
 -0.04637808 -0.06523985 -0.00366844 -0.06702595 -0.051

In [31]:
import os
import nltk
from gensim.models import Word2Vec

hypernym_file = "hypernyms.txt"

all_vectors = {}

with open(hypernym_file, "r") as file:
    text = file.read()
    words = nltk.word_tokenize(text)
    for word in words:
        try:
            word_vector = model.wv[word.lower()]
            all_vectors[word] = word_vector
            # print(f"Word: {word}\nVector: {word_vector}\n")
        except KeyError:
            continue
            # print(f"Word '{word}' not in vocabulary.\n")

print(len(all_vectors.keys()))


138


In [37]:
import numpy as np

# Convert the list to a NumPy array
vectors_array = np.array(list(all_vectors.values()))
print(vectors_array.shape)

# Compute the cosine similarity between all pairs of vectors
cosine_sim = np.dot(vectors_array, vectors_array.T)
cosine_sim /= np.linalg.norm(vectors_array, axis=1, keepdims=True)
cosine_sim /= np.linalg.norm(vectors_array, axis=1, keepdims=True).T

print(cosine_sim)

(138, 100)
[[ 0.99999994  0.03134152 -0.06566384 ...  0.01753318  0.02094397
   0.0768731 ]
 [ 0.03134153  0.99999994 -0.07252886 ...  0.08610769 -0.04806319
  -0.02930969]
 [-0.06566384 -0.07252886  1.0000001  ...  0.01336481 -0.06905692
  -0.11288552]
 ...
 [ 0.01753318  0.08610769  0.01336481 ...  0.99999976 -0.02279484
  -0.04243451]
 [ 0.02094397 -0.04806319 -0.06905693 ... -0.02279484  0.99999994
  -0.11589417]
 [ 0.0768731  -0.02930969 -0.11288552 ... -0.04243451 -0.11589417
   1.0000001 ]]


In [18]:
# # make a dictionary mapping the word to vector for each sense 
# # to be able to calculate pairwise distance


# # read through all the seed word files for each sense

import os
import numpy as np

directory_path = "./txt Files/"


sight_vectors = {}
smell_vectors = {}
sound_vectors = {}
taste_vectors = {}
touch_vectors = {}
sense_dicts = [sight_vectors, smell_vectors,
               sound_vectors, taste_vectors, touch_vectors]

sense_index = 0
for filename in os.listdir(directory_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(directory_path, filename)
        with open(file_path, "r") as file:
            text = file.read()
            words = nltk.word_tokenize(text)

            for word in words:
                curr_dict = sense_dicts[sense_index]
                curr_dict[word] = model.wv[word]


# for i, (key, value) in enumerate(sight_vectors.items()):
#     print(key, value)
#     if i == 9:
#         break



KeyError: "Key 'Hear' not present"